#### Выявляем корреляцию между длиной названия фильма и его оценкой

In [1]:
# импорт
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
# инициализация spark

spark = SparkSession \
    .builder \
    .appName("pyspark_imdb_corr") \
    .master("local[1]") \
    .config("spark.driver.memory", "10G") \
    .config("spark.jars", "postgresql-42.7.4.jar") \
    .getOrCreate()

In [3]:
# загрузка данных в датафреймы из таблиц БД

df_basics = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/imdb") \
    .option("dbtable", "basics") \
    .option("user", "postgres") \
    .option("password", "123") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_ratings = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/imdb") \
    .option("dbtable", "ratings") \
    .option("user", "postgres") \
    .option("password", "123") \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [30]:
# фильтруем тип 'movie'
df_basics = df_basics.where(df_basics.titleType == 'movie')

# join таблиц basics и ratings
# выбираем нужные колонки
# добавляем колонку с длиной названия фильма
df_join = df_basics.join(df_ratings, df_basics.tconst == df_ratings.tconst). \
    select(df_basics.primaryTitle, df_ratings.averageRating, df_ratings.numVotes). \
    withColumn("primaryTitle_length", F.length(df_basics.primaryTitle))

In [32]:
# расчет корреляции без учета количества оценок
corr_no_numVotes = df_join.select(F.corr("primaryTitle_length", "averageRating"))
corr_no_numVotes.collect()[0][0]

In [36]:
# нормализация колонки numVotes, получение конечной метрики
min = df_join.agg({"numVotes": "min"}).collect()[0][0]
max = df_join.agg({"numVotes": "max"}).collect()[0][0]
df_join_norm = df_join. \
    withColumn('norm_numVotes', (F.col("numVotes") - min)/(max - min)). \
    withColumn("final_metric", (F.col("norm_numVotes") * F.col("averageRating"))). \
    drop("norm_numVotes")

In [40]:
corr_norm = df_join_norm.select(F.corr("primaryTitle_length", "final_metric")).collect()[0][0]

In [41]:
corr_norm

-0.0134455476303509